In [1]:
!pip install alpaca

In [3]:
import sys
print(sys.executable)


c:\Users\Admin\AppData\Local\Programs\Python\Python313\python.exe


In [ ]:
import os
from dotenv import load_dotenv
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest, GetAccountRequest
from alpaca.trading.enums import AssetClass

# Ladda API-nycklar från .env-filen
load_dotenv()
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
ENV = os.getenv("ENV")

# Use paper api for development which uses virtual money instead of real
if ENV == "development":
    paper = True
else:
    paper = False

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=paper)

# search for US equities
search_params = GetAssetsRequest(asset_class=AssetClass.SV_EQUITY)

assets = trading_client.get_all_assets(search_params)

print(assets)

AttributeError: type object 'AssetClass' has no attribute 'SV_EQUITY'

In [9]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)

# search for US equities
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)

assets = trading_client.get_all_assets(search_params)

In [4]:
import pandas as pd

barset = alpaca.get_bars("AAPL", TimeFrame.Day, limit=5)
print(barset)
df = pd.DataFrame([bar.__dict__['_raw'] for bar in barset])

display(df)

[Bar({   'c': 242.92,
    'h': 244.98,
    'l': 241.42,
    'n': 312327,
    'o': 244.33,
    't': '2025-02-26T05:00:00Z',
    'v': 20299988,
    'vw': 243.013922})]


,c,h,l,n,o,t,v,vw
0,242.92,244.98,241.42,312327,244.33,2025-02-26T05:00:00Z,20299988,243.013922
